In [2]:
!ls -1 

Controller.py
Create sequence of VQs.py
Documentation.ipynb
MasterProcessor.py
diffusion_maps.py
extractPatches.py
files
run_job.py
test.py
watchdog.py
watchdog.sh


### MasterProcessor.py
defunct, replaced with "Controller"

### Controller.py

Top level controller of processing

```
usage: Controller.py [-h] scripts s3location local_data

positional arguments:
  scripts     path to the directory with the scripts
  s3location  path to the s3 directory with the lossless images
  local_data  path to the local data directory
```

In [ ]:
#!/usr/bin/env python3
import psutil
import socket
from os import getpid,mkdir,system
from subprocess import Popen,PIPE
from os.path import isfile
from glob import glob
from time import sleep,time
from os.path import isfile
from sys import argv
import re
import argparse
import numpy as np

def run(command):
    print('cmd=',command)
    system(command)
    
def runPipe(command):
    print('runPipe cmd=',command)
    p=Popen(command.split(),stdout=PIPE,stderr=PIPE)
    L=p.communicate()
    stdout=L[0].decode("utf-8").split('\n')
    stderr=L[1].decode("utf-8").split('\n')
    return stdout,stderr

def clock(message):
    print('%8.1f \t%s'%(time(),message))
    time_log.append((time(),message))

def printClock():
    t=time_log[0][0]
    for i in range(1,len(time_log)):
        print('%8.1f \t%s'%(time_log[i][0]-t,time_log[i][1]))
        t=time_log[i][0]

def list_s3_files(stack_directory):
    stdout,stderr=runPipe("aws s3 ls %s/ "%(stack_directory))
    return stdout
    
def get_file_table(stack_directory,pattern=r'(.*)\.([^\.]*)$'):
    """create a table of the files in a directory corresponding to a stack:
    stack_directory: the location of the directory on s3.
    example: s3://mousebraindata-open/MD657/
    """

    stdout = list_s3_files(stack_directory)
    pat=re.compile(pattern)

    T={}
    for file in stdout:
        parts=file.strip().split()
        if len(parts)!=4:
            continue
        filename=parts[3]
        if not 'lossless.' in filename:
            continue
        m=pat.match(filename)
        if m:
            file,ext= m.groups()
            info=(ext,parts[0]+' '+parts[1])
            if file in T:
                T[file].append(info)
            else:
                T[file]=[info]
        else:
            print(filname,'no match')
    return T

def find_and_lock(stack_directory):
    """ find a section file without a lock and lock it"""
    T=get_file_table(stack_directory)

    while True:
        #find a file without a lock
        found=False
        for item in T.items():
            if len(item[1])==1:
                found=True
                break
        if not found:
            return None
        
        filename=item[0]
        extensions=item[1]

        #create a lock
        hostname=socket.gethostname().replace('.','+')
        flagname=filename+'.lock-'+hostname
        open(scripts+'/'+flagname,'w').write(flagname+'\n')

        command='aws s3 cp %s %s/%s'%(scripts+'/'+flagname,stack_directory,flagname)
        run(command)

        # check to make sure that there is only one lock.
        T=get_file_table(stack_directory)
        extensions=T[filename]
        if len(extensions)==2:
            return filename
    
        # translation of date for better handling of two machines putting locks 
        # at nearly the same time
        # comparing the time stamps of the locks can be used to resolve who was firstand should continue
        # and who should look for another file.
        # from datetime import datetime
        # d1=datetime.strptime('2018-08-28 21:16:34','%Y-%m-%d %H:%M:%S')

def process_tiles(tile_pattern):
    i=0
    print('tile_pattern=',tile_pattern)
    for infile in glob(tile_pattern):
        stem=infile[:-4]
        #print ('infile=%s, stem=%s'%(infile,stem))
        lockfile=stem+'.lock'
        if not isfile(lockfile):
            i+=1
            print('got lock',lockfile,i)
            run('python3 {0}/run_job.py {0} {1}'.format(scripts,stem))
            sleep(0.1)
        else:
            #print('\r %s exists'%lockfile,end='')
            continue

        # Wait if load is too high
        load=np.mean(psutil.cpu_percent(percpu=True))
        print(' %5d                            load: %6.2f'%(i,load))
        j=0
        while load>85:
            print(' %5d    Sleep:%3d               load: %6.2f'%(i,j,load))
            j+=1
            sleep(2)
            load=np.mean(psutil.cpu_percent(percpu=True))

        print('\nload low enough',load)
    return i

if __name__=="__main__":
    
    parser = argparse.ArgumentParser()
    parser.add_argument("scripts", type=str,
                        help="path to the directory with the scripts")
    parser.add_argument("s3location", type=str,
                        help="path to the s3 directory with the lossless images")
    parser.add_argument("local_data",type=str,
                        help="path to the local data directory")
    args = parser.parse_args()

    scripts=args.scripts
    stack_directory=args.s3location
    local_data=args.local_data
    
    time_log=[]

    clock('starting Controller with stack_directory=%s, local_data=%s'%(stack_directory,local_data))

    try:
        #preparations: make dirs data and data/tiles
        run('sudo chmod 0777 /dev/shm/')
        mkdir(local_data)
        mkdir(local_data+'/tiles')
        clock('created data directory')
    except:
        pass


    while True:
        #find an unprocessed file on S3
        stem=find_and_lock(stack_directory)
        clock('found and locked %s'%stem)

        if stem==None:
            print('all files processed')
            break

        run('rm -rf %s/*'%(local_data))
        run('mkdir %s/tiles'%local_data)
        clock('cleaning local directory')


        #Bring in a file and break it into tiles
        run('aws s3 cp %s/%s.jp2 %s/%s.jp2'%(stack_directory,stem,local_data,stem))
        clock('copied from s3: %s'%stem)
        run('kdu_expand -i %s/%s.jp2 -o %s/%s.tif'%(local_data,stem,local_data,stem))
        clock('translated into tif')
        run('convert %s/%s.tif -crop 1000x1000  +repage  +adjoin  %s'%
            (local_data,stem,local_data)+'/tiles/tiles_%02d.tif')
        clock('broke into tiles')

        # perform analysis
        i=process_tiles('%s/tiles/tiles_*.tif'%local_data)
        clock('1 - processed %6d tiles'%i)
        i=process_tiles('%s/tiles/tiles_*.tif'%local_data)
        clock('2 - processed %6d tiles'%i)

        #copy results to s3
        run("tar czf {0}/{1}_patches.tgz {0}/tiles/*.pkl {0}/tiles/*.log {0}/tiles/*.lock".format(local_data,stem))
        clock('created tar file {0}/{1}_patches.tgz'.format(local_data,stem))

        run('aws s3 cp {0}/{1}_patches.tgz {2}/'.format(local_data,stem,stack_directory))
        clock('copy tar file to S3')

    printClock()

In [ ]:
# %load run_job.py
from sys import argv
from os import getpid,system
from os.path import isfile
from time import sleep
import datetime


def getLock(lockfile):
    try:
        l=open(lockfile, 'x')
        return l
    except FileExistsError:
        return None

scripts=argv[1]
stem=argv[2]

lockfilename=stem+'.lock'
logfilename=stem+'.log'

lockfile=getLock(lockfilename)
if not lockfile is None:

    command='python3 %s/extractPatches.py %s > %s &'%(scripts,stem,logfilename)
    # put some info into the log/lock file
    now = datetime.datetime.now()
    print('pid=',getpid(), file=lockfile)
    print('start time=',now.isoformat(), file=lockfile)
    print('command=',command, file=lockfile)
    
    system(command)
    sleep(1)
else:
    print(lockfilename,'exists, skipping')


In [ ]:
# %load watchdog.py
#!/usr/bin/env python3

from os.path import isfile,getmtime
from glob import glob
from time import sleep,time
from os import system
from subprocess import Popen,PIPE

stack='s3://mousebraindata-open/MD657'
local_data='/dev/shm/data'
exec_dir='/home/ubuntu/shapeology_code/scripts'

def runPipe(command):
    print('cmd=',command)
    p=Popen(command.split(),stdout=PIPE,stderr=PIPE)
    L=p.communicate()
    stdout=L[0].decode("utf-8").split('\n')
    stderr=L[1].decode("utf-8").split('\n')
    return stdout,stderr

def run(command,out):
    print('cmd=',command,'out=',out)
    outfile=open(out,'w')
    Popen(command.split(),stdout=outfile,stderr=outfile)

def Last_Modified(file_name):
    try:
        mtime = getmtime(file_name)
    except OSError:
        mtime = 0
    return(mtime)

if __name__=='__main__':
    Recent=False
    for logfile in glob(exec_dir+'/Controller*.log'):
        gap=time() - Last_Modified(logfile)
        if gap <120: # allow 2 minute idle
            print(logfile,'gap is %6.1f'%gap)
            Recent=True
            break
    if(not Recent):
        # Check that another 'controller' is not running
        stdout,stderr = runPipe('ps aux')
        Other_controller=False
        for line in stdout:
            if 'Controller.py' in line:
                Other_controller=True
                break
        
        if Other_controller:
            print('Other Controller.py is running')
        else:
            command='{0}/Controller.py {0} {1} {2}'\
                .format(exec_dir,stack,local_data)
            output='{0}/Controller-{1}.log'.format(exec_dir,int(time()))
            run(command,output)


In [ ]:
# %load watchdog.sh
#!/usr/bin/env bash
export PATH="/home/ubuntu/bin:/home/ubuntu/.local/bin:/home/ubuntu/KDU7A2_Demo_Apps_for_Centos7-x86-64_170827/:/home/ubuntu/shapeology_code/scripts/:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/games:/usr/local/games:/snap/bin"
export LD_LIBRARY_PATH=/home/ubuntu/KDU7A2_Demo_Apps_for_Centos7-x86-64_170827/
#echo 'this is watchdog'  >> /home/ubuntu/watchdog.log
#echo $PATH  >> /home/ubuntu/watchdog.log
echo $LD_LIBRARY_PATH >> /home/ubuntu/watchdog.log
watchdog.py >> /home/ubuntu/watchdog.log


In [ ]:
# %load extractPatches.py
import cv2
from cv2 import moments,HuMoments
import pickle
import numpy as np

def create_footprint(cell_size=41):
    center=(cell_size-1)/2.
    footprint=np.ones([cell_size,cell_size])>1

    for i in range(cell_size):
        for j in range(cell_size):
            footprint[i,j]=((i-center)**2+(j-center)**2)<=center**2
            
    ratio=sum(footprint.flatten())/(cell_size**2) # ratio of footprint area to square patch area
    return cell_size,center,ratio,footprint

def normalize_greyvals(ex):
    ex=ex*mask
    _flat=ex.flatten()
    _m=np.mean(_flat)/ratio
    _m2=np.mean(_flat**2)/ratio
    #print(_m.shape,_m2.shape,_flat.shape)
    if _m2>_m**2:
        _std=np.sqrt(_m2-_m**2)
    else:
        _std=1
        print('error in calc of _std',_m,_m2)
    #print('normalize_greyvals: mean=',_m,'std=',_std)
    ex_new=(ex-_m)/_std
    return _m,_std,ex_new * mask

def angle(ex):
    rows,cols = ex.shape
    M=moments(ex+mask)
    #print(M['m00'],M['m10'],M['m01'])
    x=M['m10']/M['m00']
    y=M['m01']/M['m00']
    nu20=(M['m20']/M['m00'])-x**2
    nu02=(M['m02']/M['m00'])-y**2
    nu11=(M['m11']/M['m00'])-y*x
    ang_est=-np.arctan(2*nu11/(nu20-nu02))/np.pi+0.5

    if ang_est>0.5:
        ang_est-=1
    ang180=(ang_est+(np.sign(nu11))/2)*90

    if ang180>=180:
        ang180-=360
    if ang180<-180:
        ang180+=360
    return ang180

def flipOrNot(ex):
    rows,cols = ex.shape
    M=moments(ex)
    x=M['m10']/M['m00'] - cols/2.
    y=M['m01']/M['m00'] - rows/2.
    if abs(x)>abs(y):
        return x<0
    else:
        return y<0


def normalize_angle(ex):
    rows,cols = ex.shape
    ang=angle(ex)
    M = cv2.getRotationMatrix2D((cols/2,rows/2),-ang,1)
    dst= cv2.warpAffine(ex,M,(cols,rows))*footprint*1
    if flipOrNot(dst):
        M180 = cv2.getRotationMatrix2D((cols/2,rows/2),180,1)
        dst = cv2.warpAffine(dst,M180,(cols,rows))

    return ang,dst*footprint*1.

from astropy.convolution import MexicanHat2DKernel,convolve
mexicanhat_2D_kernel = 10000*MexicanHat2DKernel(10)

def find_threshold(image,percentile=0.9):
    V=sorted(image.flatten())
    l=len(V)
    thr=V[int(l*percentile)] #consider only peaks in the top 5%
    return thr

def normalize(window,range=[0,1],dtype=np.float32):
    _max=max(window.flatten())
    _min=min(window.flatten())
    return np.array((window-_min)/(_max-_min),dtype=dtype)

from photutils.detection import find_peaks

def extract_patches(_mean,Peaks):
    markers=_mean*np.float32(0)
    stamp=footprint*np.float32(0.2)

    X=list(Peaks["x_peak"])
    Y=list(Peaks["y_peak"])

    extracted=[]
    for i in range(len(X)):
        corner_x=np.uint16(X[i]-center)
        corner_y=np.uint16(Y[i]-center)

        # ignore patches that extend outside of window
        if(corner_x<0 or corner_y<0 or \
           corner_x+cell_size>markers.shape[1] or corner_y+cell_size>markers.shape[0]):
            continue

        # mark location of extracted patches
        markers[corner_y:corner_y+cell_size,corner_x:corner_x+cell_size]=stamp
        # extract patch
        ex=np.array(_mean[corner_y:corner_y+cell_size,corner_x:corner_x+cell_size])
        ex *= mask

        #normalize patch interms of grey values and in terms of rotation
        _m,_std,ex_grey_normed=normalize_greyvals(ex)
        rot_angle1,ex_rotation_normed=normalize_angle(ex_grey_normed)
        rot_angle2,ex_rotation_normed=normalize_angle(ex_rotation_normed)
        extracted.append((_m,_std,rot_angle1+rot_angle2,ex_rotation_normed*mask))
    return extracted,markers
    
def check_blank(window,min_std=10):
    # find whether window mosly blank and should be ignored.
    return np.std(window.flatten()) < min_std

def preprocess(window):
    _mean=normalize(window)
    P=convolve(_mean,mexicanhat_2D_kernel)
    thr=find_threshold(P,0.9)
    Peaks=find_peaks(P,thr,footprint=footprint)
    return _mean,P,Peaks

if __name__=="__main__":

    import argparse
    from time import time
    parser = argparse.ArgumentParser()
    parser.add_argument("filestem", type=str,
                    help="Process <filestem>.pkl into <filestem>_extracted.pkl")

    args = parser.parse_args()
    infile = args.filestem+'.tif'
    outfile= args.filestem+'_extracted.pkl'

    window=cv2.imread(infile,cv2.IMREAD_GRAYSCALE | cv2.IMREAD_ANYDEPTH)

    if check_blank(window):
        print('image',infile,'too blank, skipping')
    else:
        t0=time()
        print('processing',infile,'into',outfile)
        cell_size,center,ratio,footprint=create_footprint(cell_size=41)
        mask=1.*footprint
        _mean,P,Peaks=preprocess(window)    
        print('found',len(Peaks),'patches')
        extracted,markers=extract_patches(_mean,Peaks)

        pickle.dump(extracted,open(outfile,'wb'))
        print('finished in %5.1f seconds'%(time()-t0))


In [ ]:
# %load Create sequence of VQs.py
import pickle as pk
import numpy as np
from numpy import *
from glob import glob
from subprocess import Popen,PIPE
from os import system
from os.path import isfile
import matplotlib.pyplot as plt
from astropy.convolution import Gaussian2DKernel,convolve

def run(command):
    print('cmd=',command)
    system(command)
    
def runPipe(command):
    print('runPipe cmd=',command)
    p=Popen(command.split(),stdout=PIPE,stderr=PIPE)
    L=p.communicate()
    stdout=L[0].decode("utf-8").split('\n')
    stderr=L[1].decode("utf-8").split('\n')
    return stdout,stderr

def clock(message):
    print('%8.1f \t%s'%(time(),message))
    time_log.append((time(),message))

def printClock():
    t=time_log[0][0]
    for i in range(1,len(time_log)):
        print('%8.1f \t%s'%(time_log[i][0]-t,time_log[i][1]))
        t=time_log[i][0]

def list_s3_files(stack_directory):
    stdout,stderr=runPipe("aws s3 ls %s/ "%(stack_directory))
    filenames=[]
    for line in stdout:
        parts=line.strip().split()
        if len(parts)!=4:
            continue
        filenames.append(parts[-1])
    return filenames

def read_files(s3_dir,_delete=False):
    s3files=list_s3_files(s3_dir)
    for filename in s3files:
        if not isfile(data_dir+'/'+filename):
            run('aws s3 cp %s/%s %s'%(s3_dir,filename,data_dir))
        D=fromfile(data_dir+'/'+filename,dtype=np.float16)
        pics=D.reshape([-1,_size,_size])
        if _delete:
            run('rm %s/%s'%(data_dir,filename))
        yield pics

def data_stream(s3_dir='s3://mousebraindata-open/MD657/permuted'):
    for pics in read_files(s3_dir):
        j=0
        for i in range(pics.shape[0]):
            if j%1000==0:
                print('\r examples read=%10d'%j,end='')
            j+=1    
            yield pics[i,:,:]

def calc_err(pic,gaussian = Gaussian2DKernel(1,x_size=7,y_size=7)):
    factor=sum(gaussian)
    P=convolve(pic,gaussian)/factor
    error=sqrt(mean(abs(pic-P)))
    sub=P[::2,::2]
    return error,sub

def plot_patches(data,h=40,w=15,_titles=[]):
    figure(figsize=(w*2,h*2))
    for i in range(h*w):
        if i>=data.shape[0]:
            break
        subplot(h,w,i+1);
        pic=np.array(data[i,:,:],dtype=np.float32)

        fig=imshow(pic,cmap='gray')
        if(len(_titles)>i):
            plt.title(_titles[i])
        fig.axes.get_xaxis().set_visible(False)
        fig.axes.get_yaxis().set_visible(False)

def pack_pics(Reps):
    size=Reps[0].shape[0]
    _len=len(Reps)
    Reps_mat=np.zeros([_len,size,size])
    for i in range(_len):
        Reps_mat[i,:,:]=Reps[i]
    return Reps_mat

def dist2(a,b):
    diff=(a-b)**2
    return sum(diff.flatten())

def dist_hist(data):
    D=[]
    for i in range(1,data.shape[0]):
        D.append(dist2(data[i,:,:],data[i-1,:,:]))
        if i%1000==0:
            print('\r',i,end='')
    hist(D,bins=100);

def refineKmeans(data_stream,Reps,per_rep_sample=100,refinement_iter=3):
    _shape=Reps[0].shape
    new_Reps=[np.zeros(_shape) for r in Reps]
    _area=_shape[0]*_shape[1]
    Reps_count=[0.0 for r in Reps]
    error=0
    count=per_rep_sample*len(Reps)
    i=0
    for patch in data_stream: 
        dists=[dist2(patch,r) for r in Reps]
        _argmin=argmin(dists)
        _min=min(dists)
        new_Reps[_argmin]+=patch
        Reps_count[_argmin]+=1
        error+=_min
        i+=1
        if i >= count:
            break
    error /= (count*_area)
    final_Reps=[]
    final_counts=[]
    for i in range(len(new_Reps)):
        if Reps_count[i]>refinement_iter:
            final_Reps.append(new_Reps[i]/Reps_count[i])
            final_counts.append(Reps_count[i])
    return final_Reps,final_counts,error

def Kmeans(data_stream,Reps=[],n=100,scale=550):
    Reps,Statistics = Kmeanspp(data_stream,Reps,n,scale)
    for i in range(5):
        Reps,final_counts,error = refineKmeans(data_stream,Reps)
        print('refine iteration %2d, error=%7.3f, n_Reps=%5d'%(i,error,len(Reps)))
    return Reps,final_counts

def Kmeanspp(data_stream,Reps=[],n=100,scale=550):
    if len(Reps)==0:
        Reps=[next(data_stream)]

    Statistics=[]
    j=len(Reps)
    for patch in data_stream: 
        _min=100000
        for r in Reps:
            _min=min(_min,dist2(patch,r))
        Prob=_min/scale
        print('\r','i=%10d,  #reps=%10d  Prob=%8.6f'%(i,len(Reps),Prob),end='')
        Statistics.append((i,len(Reps),_min))
        if np.random.rand()<Prob:
            Reps.append(patch)
            j+=1
        if j>=n:
            break
    return Reps,Statistics

def plot_statistics(Statistics,alpha=0.05,_start=10): 
    N=[x[1] for x in Statistics]
    d=[x[2] for x in Statistics]

    s=mean(d[:_start])
    smoothed=[s]*_start
    for x in d[_start:]:
        s=(1-alpha)*s + alpha*x
        smoothed.append(s)
    loglog(N[_start:],smoothed[_start:])
    xlabel('N')
    ylabel('smoothed distance')
    grid(which='both')


def filtered_images(s3_dir='s3://mousebraindata-open/MD657/permuted',reduce_res=True,smooth_threshold=0.4):
    for pic in data_stream(s3_dir):
        err,sub=calc_err(pic)
        if err>smooth_threshold:
            continue
        if reduce_res:
            yield sub
        else:
            yield pic


gen=filtered_images(smooth_threshold=0.35,reduce_res=False)
Reps,final_count=Kmeans(gen,n=10)
plot_patches(pack_pics(Reps),_titles=['%4d'%x for x in final_count])



In [ ]:
# %load diffusion_maps.py
import pickle as pk
import numpy as np
from numpy import *
from glob import glob
from subprocess import Popen,PIPE
from os import system
from os.path import isfile
import matplotlib.pyplot as plt
from astropy.convolution import Gaussian2DKernel,convolve

# ## Calculate laplacian random-walk matrix
# 
# The matrix corresponds to a simple random walk on the individual examples  where the location of each example is replaced by the location of the corresponding representative. 
# 
# We use [pydiffmap](https://pydiffmap.readthedocs.io/en/master/)

# In[84]:


L=len(new_Reps)
new_Reps[0].shape
data1D=np.concatenate([x.reshape([1,441]) for x in new_Reps])
data1D.shape


# In[85]:


dists=[]
for i in range(L):
    for j in range(i):
        dists.append(dist2(new_Reps[i],new_Reps[j]))
hist(dists);
title('distances between centroids')


# In[ ]:


get_ipython().system('sudo pip3 install pydiffmap')


# In[86]:


data1D.shape,len(Reps_count)


# In[87]:




from pydiffmap import diffusion_map as dm
# initialize Diffusion map object.
neighbor_params = {'n_jobs': -1, 'algorithm': 'ball_tree'}

mydmap = dm.DiffusionMap(n_evecs=50, k=20, epsilon=100.0, alpha=1.0, neighbor_params=neighbor_params)
# fit to data and return the diffusion map.
dmap = mydmap.fit_transform(data1D,weights=Reps_count)

#%pylab inline
pylab.scatter(dmap[:,0],dmap[:,1]);


# In[88]:


hist(Reps_count);
title('no. of examples per partition')


# In[89]:


image_size=np.array(new_Reps[0].shape)
canvas_size=np.array([1000,1000])
_minx=min(dmap[:,0])
_maxx=max(dmap[:,0])
_miny=min(dmap[:,1])
_maxy=max(dmap[:,1])
shift_x = -_minx
scale_x = canvas_size[0]/(_maxx - _minx)
shift_y = -_miny
scale_y = canvas_size[1]/(_maxy - _miny)

x=[int((_x+shift_x)*scale_x) for _x in dmap[:,0]]
y=[int((_y+shift_y)*scale_y) for _y in dmap[:,1]]

canvas=2*np.ones(canvas_size+image_size)
for i in range(len(new_Reps)):
    if(Reps_count[i]>30):
        canvas[x[i]:x[i]+image_size[0],y[i]:y[i]+image_size[1]]=new_Reps[i]


# In[90]:


figure(figsize=[40,40])
imshow(canvas,cmap='gray')


# In[ ]:


max(new_Reps[0].flatten())


# In[ ]:


from pydiffmap.visualization import embedding_plot, data_plot

embedding_plot(mydmap, scatter_kwargs = {'c': dmap[:,0], 'cmap': 'Spectral'})
data_plot(mydmap, dim=3, scatter_kwargs = {'cmap': 'Spectral'})

plt.show()


# In[ ]:


A=np.zeros([L,L])
for i in range(L):
    for j in range(L):
        w=exp(-dist2(new_Reps[i],new_Reps[j])/sigma2)
        A[i,j]=w * Reps_count[i]*Reps_count[j]


# In[ ]:


D=sum(A,axis=0)
D2=diag(1/sqrt(D))

NA=np.dot(D2,np.dot(A,D2))

w,v = np.linalg.eig(NA)
hist(NA.flatten(),bins=100);


# In[ ]:


i=0
print('eig no %3d eigval=%5.3f'%(i,w[i]))
sorted_v=sort(v[i,:])
order=argsort(v[i,:])
plot_patches(Reps_mat[order],h=10,w=10,_titles=['c_%1d=%6.3f'%(i,x) for x in sorted_v])

for i in range(L):
    for j in range(L):
        print(' %0.1f'%v[i,j],end='')
    print('')
# In[ ]:


plot(w[:10])


# In[ ]:


scatter(v[1,:],v[2,:])


# In[ ]:


v[2,:]



def run(command):
    print('cmd=',command)
    system(command)
    
def runPipe(command):
    print('runPipe cmd=',command)
    p=Popen(command.split(),stdout=PIPE,stderr=PIPE)
    L=p.communicate()
    stdout=L[0].decode("utf-8").split('\n')
    stderr=L[1].decode("utf-8").split('\n')
    return stdout,stderr

def clock(message):
    print('%8.1f \t%s'%(time(),message))
    time_log.append((time(),message))

def printClock():
    t=time_log[0][0]
    for i in range(1,len(time_log)):
        print('%8.1f \t%s'%(time_log[i][0]-t,time_log[i][1]))
        t=time_log[i][0]

def list_s3_files(stack_directory):
    stdout,stderr=runPipe("aws s3 ls %s/ "%(stack_directory))
    filenames=[]
    for line in stdout:
        parts=line.strip().split()
        if len(parts)!=4:
            continue
        filenames.append(parts[-1])
    return filenames


# In[14]:


s3_dir='s3://mousebraindata-open/MD657/permuted'
permuted_files=list_s3_files(s3_dir)
permuted_files[:10]


# In[26]:


get_ipython().system('touch $data_dir/tmp')
get_ipython().system('ls $data_dir')
isfile(data_dir+'/tmp')


# In[27]:


def read_files(s3_dir,_delete=False):
    s3files=list_s3_files(s3_dir)
    for filename in s3files:
        if not isfile(data_dir+'/'+filename):
            run('aws s3 cp %s/%s %s'%(s3_dir,filename,data_dir))
        D=fromfile(data_dir+'/'+filename,dtype=np.float16)
        pics=D.reshape([-1,_size,_size])
        if _delete:
            run('rm %s/%s'%(data_dir,filename))
        yield pics


# In[28]:


def data_stream():
    for pics in read_files('s3://mousebraindata-open/MD657/permuted'):
        for i in range(pics.shape[0]):
            yield pics[i,:,:]


# In[34]:


get_ipython().system('ls -lrt ../../data')


# In[64]:


pics_list=[]
i=0
for pic in data_stream():
    pics_list.append(np.array(pic,dtype=np.float32))
    i+=1
    if i>=300000:
        break

factor=sum(gaussian = Gaussian2DKernel(1,x_size=7,y_size=7))
print('factor=',factor)
def calc_err(pic):
    P=convolve(pic,gaussian)/factor
    error=sqrt(mean(abs(pic-P)))
    sub=P[::2,::2]
    return error,sub


# In[66]:


def plot_patches(data,h=40,w=15,_titles=[]):
    figure(figsize=(w*2,h*2))
    for i in range(h*w):
        if i>=data.shape[0]:
            break
        subplot(h,w,i+1);
        pic=data[i,:,:]
        #P=convolve(pic,gaussian)/factor

        fig=imshow(pic,cmap='gray')
        if(len(_titles)>i):
            plt.title(_titles[i])
        fig.axes.get_xaxis().set_visible(False)
        fig.axes.get_yaxis().set_visible(False)
#plot_patches(scombined,h=2,_titles=[str(i) for i in range(scombined.shape[0])])


# In[67]:


def pack_pics(Reps):
    size=Reps[0].shape[0]
    _len=len(Reps)
    Reps_mat=np.zeros([_len,size,size])
    for i in range(_len):
        Reps_mat[i,:,:]=Reps[i]
    return Reps_mat


# In[68]:


pics=pack_pics(pics_list)


# In[69]:


plot_patches(pics)


# In[70]:


shift=1000
selected=[]
errors=[]
i=0; j=0;
while j < 100:
    pic=pics[i+shift,:,:]
    i+=1
    error,sub=calc_err(pic)
    if error<0.4:
        continue
    j+=1
    selected.append(pic)
    errors.append('%4.2f'%error)
plot_patches(pack_pics(selected),h=10,w=10,_titles=errors)


# In[71]:


#collect images that are pretty smooth
# reduce resolution by a factor of 2
low_err=[]
for i in range(pics.shape[0]):
    pic=pics[i,:,:]
    error,sub=calc_err(pic)
    if error<0.4:
        low_err.append(sub)
        j=len(low_err)
        if j%1000==0:
            print('\r',i,j,end='')

lcombined=np.stack(low_err)
lcombined.shape


# In[72]:


lcombined.shape


# In[73]:


def dist2(a,b):
    diff=(a-b)**2
    return sum(diff.flatten())


# In[74]:


D=[]
for i in range(1,lcombined.shape[0]):
    D.append(dist2(lcombined[i,:,:],lcombined[i-1,:,:]))
    if i%1000==0:
        print('\r',i,end='')


# In[75]:


hist(D,bins=100);


# In[76]:


max(D)


# In[77]:


def refineKmeans(data,Reps):
    new_Reps=[np.zeros(Reps[0].shape) for r in Reps]
    Reps_count=[0.0 for r in Reps]
    error=0
    for i in range(data.shape[0]): 
        patch=data[i,:,:]
        dists=[dist2(patch,r) for r in Reps]
        _argmin=argmin(dists)
        _min=min(dists)
        new_Reps[_argmin]+=patch
        Reps_count[_argmin]+=1
        error+=_min
    error /= data.shape[0]
    final_Reps=[]
    final_counts=[]
    for i in range(len(new_Reps)):
        if Reps_count[i]>5:
            final_Reps.append(new_Reps[i]/Reps_count[i])
            final_counts.append(Reps_count[i])
    return final_Reps,final_counts,error


# In[78]:


def Kmeans(data,n=100,scale=550):
    Reps,Statistics = Kmeanspp(data,n,scale)
    for i in range(5):
        Reps,error = refineKmeans(data,Reps)
        print('refine iteration %2d, error=%7.3f'%(i,error))


# In[79]:


def Kmeanspp(data,n=100,scale=550):
    Reps=[data[0,:,:]]

    Statistics=[]
    j=1
    for i in range(1,data.shape[0]): 
        _min=100000
        patch=data[i,:,:]
        for r in Reps:
            _min=min(_min,dist2(patch,r))
        Prob=_min/scale
        print('\r','i=%10d,  #reps=%10d  Prob=%8.6f'%(i,len(Reps),Prob),end='')
        Statistics.append((i,len(Reps),_min))
        if np.random.rand()<Prob:
            Reps.append(patch)
            j+=1
        if j>=n:
            break
    return Reps,Statistics


# In[80]:


def plot_statistics(Statistics,alpha=0.05,_start=10): 
    N=[x[1] for x in Statistics]
    d=[x[2] for x in Statistics]

    s=mean(d[:_start])
    smoothed=[s]*_start
    for x in d[_start:]:
        s=(1-alpha)*s + alpha*x
        smoothed.append(s)
    loglog(N[_start:],smoothed[_start:])
    xlabel('N')
    ylabel('smoothed distance')
    grid(which='both')


# In[81]:


N=300

Reps, Statistics = Kmeanspp(lcombined,n=N)
Reps_mat = pack_pics(Reps)
plot_patches(Reps_mat,h=5,w=10)


# In[82]:


Reps_mat.shape


# In[83]:


for i in range(1,4):
    new_Reps,Reps_count,error = refineKmeans(lcombined[i*10000:(i+1)*10000,:,:],Reps)
    print(i,error,len(Reps_count))
    Reps_mat = pack_pics(new_Reps)
    plot_patches(Reps_mat,h=1,w=10,_titles=['%4d'%x for x in Reps_count])
    Reps=new_Reps
plot_patches(Reps_mat,h=10,w=10,_titles=['final_%4d'%x for x in Reps_count])


# ## Calculate laplacian random-walk matrix
# 
# The matrix corresponds to a simple random walk on the individual examples  where the location of each example is replaced by the location of the corresponding representative. 
# 
# We use [pydiffmap](https://pydiffmap.readthedocs.io/en/master/)

# In[84]:


L=len(new_Reps)
new_Reps[0].shape
data1D=np.concatenate([x.reshape([1,441]) for x in new_Reps])
data1D.shape


# In[85]:


dists=[]
for i in range(L):
    for j in range(i):
        dists.append(dist2(new_Reps[i],new_Reps[j]))
hist(dists);
title('distances between centroids')


# In[ ]:


get_ipython().system('sudo pip3 install pydiffmap')


# In[86]:


data1D.shape,len(Reps_count)


if __name__=="__main__":
    _size=41
    data_dir="../../data"

    

